import stuff

In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/SSNN-BP')

tf.nn.depthwise_conv2d will be used to simulate the SNN with a Spike Response Model. For this, it needs input of shape (batch, 1, in_width, in_channels) and filters of shape (1, filter_width, in_channels, 1). The convolution occurs in the dimension of width - which should correspond to simulation steps
Describe the model in the following block

In [ ]:
source_path = '/content/drive/MyDrive/Colab Notebooks/SSNN-BP/'

In [ ]:
import matplotlib.pyplot as plt

X_train_load = np.load(source_path + 'train_LSM_out_NMNIST_p2.npy')
y_train_load = np.load(source_path + 'train_labels_NMNIST_p2.npy')
print("max = ", np.max(X_train_load))
print("min = ", np.min(X_train_load))
print("mean = ", np.mean(X_train_load))

X_test_load = np.load(source_path + 'test_LSM_out_NMNIST_p2.npy')
y_test_load = np.load(source_path + 'test_labels_NMNIST_p2.npy')

In [ ]:
tf.disable_v2_behavior()
seed = 0
rng = np.random.RandomState(seed)
tf.set_random_seed(seed)
np.seterr(all='raise')

batch_size = 100
# SNN parameters

data_size = 60000
data_size_test = 8000

num_iter = 51
max_in_spikes = 51
max_out_spikes = 25
boost_factor = 1

X_train_load = boost_factor*X_train_load
X_train_load[X_train_load>1] = 1

X_test_load = boost_factor*X_test_load
X_test_load[X_test_load>1] = 1

print("total train size: ", X_train_load.shape[0])
print("total test size: ", X_test_load.shape[0])
print("train size: ", data_size)
print("test size: ", data_size_test)

N = X_train_load.shape[1]
num_output = 10

X_pl = tf.placeholder('float', [batch_size, 1, num_iter, N])

X_data = X_train_load[:data_size]
y_data = y_train_load[:data_size]

X_test = X_test_load[-data_size_test:]
#X_test = X_test + np.random.normal(0, 0.1, (X_test.shape[0], X_test.shape[1]))
y_test = y_test_load[-data_size_test:]

# prepare the inputs
X_in_train = np.zeros((X_data.shape[0], 1, num_iter, X_data.shape[1]), dtype=np.int8)
y_in_train = np.zeros((X_data.shape[0], 1, num_iter, num_output), dtype=np.int8)
X_in_test = np.zeros((X_test.shape[0], 1, num_iter, X_test.shape[1]), dtype=np.int8)
y_in_test = np.zeros((X_test.shape[0], 1, num_iter, num_output), dtype=np.int8)

for i in range(X_data.shape[0]):
    label_ind = np.int32(y_data[i])
    spikes = np.random.uniform(size=num_iter)
    spikes[spikes<(max_out_spikes/num_iter)] = 1
    spikes[spikes<1] = 0
    y_in_train[i,0,:,label_ind] = spikes

for i in range(X_test.shape[0]):
    label_ind = np.int32(y_test[i])
    spikes = np.random.uniform(size=num_iter)
    spikes[spikes<(max_out_spikes/num_iter)] = 1
    spikes[spikes<1] = 0
    y_in_test[i,0,:,label_ind] = spikes

In [ ]:
batch_size = 100

X_in_train = np.zeros((X_data.shape[0], 1, num_iter, X_data.shape[1]), dtype=np.int8)
X_in_test = np.zeros((X_test.shape[0], 1, num_iter, X_test.shape[1]), dtype=np.int8)

num_batches = data_size//batch_size
num_batches_test = data_size_test//batch_size
for b in range(num_batches):
  if(b%50==0):
    print("completed: ", b/num_batches)
  batch_data = X_data[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)
  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1, 1)) #shape=(batch_size, N, 1, 1)
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter, 1)) #shape=(batch_size, N, num_iter, 1)
  batch_data_trans = np.transpose(batch_data_tiled, (0,3,2,1)) #shape=(batch_size, 1, num_iter, N)
  batch_in = np.random.uniform(size=(batch_size, 1, num_iter, N))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0
  X_in_train[b*batch_size:(b+1)*batch_size] = batch_in

for b in range(num_batches_test):
  batch_data = X_test[b*batch_size:(b+1)*batch_size]*(max_in_spikes/num_iter)
  batch_reshp = np.reshape(batch_data, (batch_size, -1, 1, 1))
  batch_data_tiled = np.tile(batch_reshp, (1, 1, num_iter, 1))
  batch_data_trans = np.transpose(batch_data_tiled, (0,3,2,1))
  batch_in = np.random.uniform(size=(batch_size, 1, num_iter, N))
  batch_in[batch_in<batch_data_trans] = 1
  batch_in[batch_in<1] = 0
  X_in_test[b*batch_size:(b+1)*batch_size] = batch_in

In [ ]:
import RateSNN as snn

num_epochs = 101
batch_size = 200
lr = 0.1
beta = 0.9
eps = 1e-8
k_size = num_iter
hidden_size = 1280
th_val_h = 5
th_val_y = 5

self_iter = 5
#self_iter = 0

# Initialize Placeholders (X_in, labels, dropout_masks, kernels)
keep_prob1 = tf.placeholder('float', shape=())
keep_prob2 = tf.placeholder('float', shape=())
tf_kernelh = tf.placeholder('float', [1, k_size, hidden_size, 1])
tf_kernelh_b = tf.placeholder('float', [1, k_size, hidden_size, 1])
tf_kernely = tf.placeholder('float', [1, k_size, num_output, 1])
tf_kernely_b = tf.placeholder('float', [1, k_size, num_output, 1])
dropout_mask1 = tf.placeholder('float', [batch_size,1,num_iter,N])
dropout_mask2 = tf.placeholder('float', [batch_size,1,num_iter,hidden_size])
X_pl = tf.placeholder('float', [batch_size, 1, num_iter, N])
s_label = tf.placeholder('float', [batch_size, 1, num_iter, num_output])

# Initialize Weights (He initialization)
W1 = tf.Variable(tf.random_normal((N,hidden_size), stddev=np.sqrt(4/N)), trainable=False)
W2 = tf.Variable(tf.random_normal((hidden_size,num_output), stddev=np.sqrt(4/hidden_size)), trainable=False)
W_inh = tf.Variable(tf.matrix_set_diag(1.0*tf.ones((num_output, num_output)), tf.zeros(num_output)), trainable=False)
W1mt = tf.Variable(tf.zeros((N,hidden_size)), trainable=False)
W2mt = tf.Variable(tf.zeros((hidden_size, num_output)), trainable=False)

W1mt_acc = tf.Variable(W1mt)
W2mt_acc = tf.Variable(W2mt)

# MOMENTUM
#W1_acc = W1
#W2_acc = W2

# NESTEROV ACCELERATED GRADIENT
W1_acc = W1 + beta*W1mt_acc
W2_acc = W2 + beta*W2mt_acc

X_masked, s_out_h_acc = snn.forward_pass(X_pl, dropout_mask1, W1_acc, th_val_h, tf_kernelh)
s_out_h_masked, s_out_y_acc = snn.y_WTA(s_out_h_acc, dropout_mask2, W2_acc, th_val_y, tf_kernely, W_inh, self_iter)

del_p_y, del_m_y, W2gt_orig = snn.finalW_update(s_out_y_acc, s_out_h_masked, s_label, th_val_y, tf_kernely_b)
del_p_h, del_m_h, W1gt_orig = snn.innerW_update(s_out_h_masked, X_masked, del_p_y, del_m_y, W2_acc, th_val_h, tf_kernelh_b)

W1mt_acc = beta*W1mt_acc + lr*W1gt_orig/batch_size
W2mt_acc = beta*W2mt_acc + lr*W2gt_orig/batch_size

new_W1 = W1.assign_add(W1mt_acc)
new_W2 = W2.assign_add(W2mt_acc)

# define the SRM kernels here
tau = num_iter/10
kernelh = snn.gen_kernel(k_size, hidden_size, tau)
kernely = snn.gen_kernel(k_size, num_output, tau)

tau = num_iter/100
kernely_b = snn.gen_kernel(k_size, num_output, tau)
kernelh_b = snn.gen_kernel(k_size, hidden_size, tau)

Run the training

In [ ]:
gpu_opts = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
pr_fq = 2
all_test_accuracies = []
all_test_accuracies_acc = []
max_acc = 0
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_opts)) as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs):
        num_batches = data_size//batch_size
        print("epoch : ", e)
        accs = []
        rand_perm = rng.permutation(data_size)
        X_in_train = X_in_train[rand_perm]
        y_in_train = y_in_train[rand_perm]
        pick_dropout_prob = np.random.uniform()
        dropout_prob2 = 0.3
        dropout_prob1 = 0.2
        for b in range(num_batches):
            #generate dropout mask
            p = np.random.uniform(size=N)
            p[p<dropout_prob1] = 0
            p[p>0] = 1
            ps_arr1 = np.zeros((batch_size, 1, num_iter, N))
            ps_arr1[:, :, :, p>0] = 1

            p = np.random.uniform(size=hidden_size)
            p[p<dropout_prob2] = 0
            p[p>0] = 1
            ps_arr2 = np.zeros((batch_size, 1, num_iter, hidden_size))
            ps_arr2[:, :, :, p>0] = 1

            in_batch = np.float64(X_in_train[b*batch_size:(b+1)*batch_size])
            label_batch = np.float64(y_in_train[b*batch_size:(b+1)*batch_size])
            feeds = {X_pl: in_batch, s_label: label_batch, tf_kernelh: kernelh, tf_kernelh_b: kernelh_b, tf_kernely: kernely, tf_kernely_b: kernely_b, dropout_mask1: ps_arr1, dropout_mask2: ps_arr2, keep_prob1: 1-dropout_prob1, keep_prob2: 1-dropout_prob2}
            out_y,_,_ = sess.run([s_out_y_acc, new_W1, new_W2], feed_dict=feeds)

            out_means = np.mean(out_y, axis=(1,2))
            label_means = np.mean(label_batch, axis=(1,2))
            out_labels = np.argmax(out_means, axis=1)
            orig_labels = np.argmax(label_means, axis=1)
            batch_acc = np.mean(np.float32(out_labels==orig_labels))
            accs.append(batch_acc)
        if e%pr_fq == 0:
            test_accs = []
            test_accs_acc = []
            num_test_batches = data_size_test//batch_size
            for c in range(num_test_batches):
                in_batch = np.float64(X_in_test[c*batch_size:(c+1)*batch_size])
                label_batch = np.float64(y_in_test[c*batch_size:(c+1)*batch_size])
                ps_arr_test1 = np.ones((batch_size, 1, num_iter, N))
                ps_arr_test2 = np.ones((batch_size, 1, num_iter, hidden_size))
                feeds = {X_pl: in_batch, tf_kernelh: kernelh, tf_kernely: kernely, dropout_mask1: ps_arr_test1, dropout_mask2: ps_arr_test2, keep_prob1: 1, keep_prob2: 1}
                out_h,out_y = sess.run([s_out_h_acc, s_out_y_acc], feed_dict=feeds)

                out_means = np.mean(out_y, axis=(1,2))
                label_means = np.mean(label_batch, axis=(1,2))
                out_labels = np.argmax(out_means, axis=1)
                orig_labels = np.argmax(label_means, axis=1)
                batch_acc = np.mean(np.float32(out_labels==orig_labels))
                test_accs.append(batch_acc)
            test_accuracy = np.mean(batch_size*np.array(test_accs))/batch_size
            all_test_accuracies.append(test_accuracy)
            print("test accuracy : ", test_accuracy)
        epoch_accuracy = np.mean(batch_size*np.array(accs))/batch_size
        print("accuracy : ", epoch_accuracy)
    final_W1 = W1.eval()
    final_W2 = W2.eval()

In [ ]:

plt.figure()
plt.plot(np.arange(0,2*len(all_test_accuracies),2), 100*np.array(all_test_accuracies))
plt.xlabel('epochs')
plt.ylabel('test accuracy (%)')
plt.show()
print("n_iter : ", num_iter)
print("max test acc : " + str(np.max(np.array(all_test_accuracies))))
print("last test acc : " + str(all_test_accuracies[-1]))
print("H1 : ", hidden_size)
#print(all_test_accuracies)
running_avg_acc = np.zeros_like(np.array(all_test_accuracies))
running_std_acc = np.zeros_like(np.array(all_test_accuracies))
for i in range(len(all_test_accuracies)):
    if i<9:
        running_avg_acc[i] = np.mean(np.array(all_test_accuracies)[0:i+1])
        running_std_acc[i] = np.std(np.array(all_test_accuracies)[0:i+1])
    else:
        running_avg_acc[i] = np.mean(np.array(all_test_accuracies)[i-9:i+1])
        running_std_acc[i] = np.std(np.array(all_test_accuracies)[i-9:i+1])

plt.figure()
plt.plot(np.arange(0,2*len(all_test_accuracies),2), 100*running_avg_acc)
plt.xlabel('epochs')
plt.ylabel('running average (last 10) of test accuracy (%)')

print('self iter - ', self_iter)
print('dropout 1 - ', dropout_prob1)
print('dropout 2 - ', dropout_prob2)
print('raw test accuracies')
print(all_test_accuracies)
print('running avg test accuracies')
print(running_avg_acc)
print('running std test accuracies')
print(running_std_acc)


In [ ]:
print("number of input  spikes per sample : ", np.sum(in_batch)/batch_size)
print("number of hidden spikes per sample : ", np.sum(out_h)/batch_size)
print("number of output spikes per sample : ", np.sum(out_y)/batch_size)
print("number of target spikes per sample : ", np.sum(label_batch)/batch_size)
#print("number of output spikes per sample w/o inh : ", np.sum(out_y_orig)/100)